# Run Obstacle Heuristic on Voxel Grid


In [1]:
import cupoch as cph
import numpy as np
import cupy as cp
import time

import numba
from numba import cuda


In [2]:
## https://github.com/mikegrudic/TreePotential/blob/master/TreePotential.py#L143

In [3]:

class PointCloudVisualizer:
    def __init__(self, width=650, height=650):
        self.width = width
        self.height = height
        
    def setup_plotly(self):
        import plotly.io as pio
        pio.templates.default = "plotly_white"
        
    def visualize_points(self, points):
        import plotly.graph_objects as go
        self.setup_plotly()

        fig = go.Figure(data=[go.Scatter3d(
            x=points[:,0],
            y=points[:,1], 
            z=points[:,2],
            mode='markers',
            marker=dict(
                size=2,
                color=points[:,2],    # Use z-coordinates for color
                colorscale='Turbo',   # Add colorscale
                opacity=0.8,
                colorbar=dict(title='Z') # Add colorbar with title
            )
        )])

        fig.update_layout(
            scene=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='Z'
            ),
            title='3D Point Cloud',
            width=self.width,
            height=self.height,
            margin=dict(l=0, r=0, t=30, b=0)
        )

        fig.show()
    
    def visualize_voxel_grid(self, voxel_grid):
        import plotly.graph_objects as go
        self.setup_plotly()
        
        # Get voxel coordinates and colors
        voxels = voxel_grid.voxels.cpu()
        voxel_coords = []
        voxel_colors = []

        for voxel in voxels.values():
            voxel_coords.append(voxel.grid_index)
            voxel_colors.append(voxel.color)

        voxel_coords = np.array(voxel_coords)
        voxel_colors = np.array(voxel_colors)

        # Create plotly figure
        fig = go.Figure(data=[go.Scatter3d(
            x=voxel_coords[:,0],
            y=voxel_coords[:,1], 
            z=voxel_coords[:,2],
            mode='markers',
            marker=dict(
                size=5,
                color=voxel_coords[:,2],  # Use z-coordinate for coloring
                colorscale='turbo',
                opacity=0.8
            )
        )])

        fig.update_layout(
            scene=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='Z'
            ),
            title='3D Voxel Grid',
            width=self.width,
            height=self.height,
            margin=dict(l=0, r=0, t=30, b=0)
        )

        fig.show()

In [4]:
class DatasetGenerator:
    def __init__(self, bbox_min=np.array([0, 0, 0]), bbox_max=np.array([50, 50, 50]), n_points=10000, seed=42):
        self.bbox_min = bbox_min
        self.bbox_max = bbox_max
        self.n_points = n_points
        self.voxel_size = 1.0
        self.min_bound = bbox_min
        self.max_bound = bbox_max
        self.scene_bbox = cph.geometry.AxisAlignedBoundingBox(self.min_bound, self.max_bound)

        np.random.seed(seed)
        
        
    def generate_ground_points(self):
        # Create a ground plane with some noise
        ground_z = np.random.normal(5, 0.5, (self.n_points//2, 1))  # Ground at z=5 with noise
        ground_xy = np.random.rand(self.n_points//2, 2) * (self.bbox_max[:2] - self.bbox_min[:2]) + self.bbox_min[:2]
        return np.hstack([ground_xy, ground_z])
        
    def generate_structure_points(self, num_structures=10):
        structure_points = []
        for _ in range(num_structures):
            # Random position for the structure
            base_x = np.random.uniform(self.bbox_min[0], self.bbox_max[0])
            base_y = np.random.uniform(self.bbox_min[1], self.bbox_max[1])
            
            # Generate points along the height
            height = np.random.uniform(20, 40)
            num_points = self.n_points//(2*num_structures)
            structure_z = np.random.uniform(5, height, (num_points, 1))
            structure_xy = np.random.normal(loc=[base_x, base_y], scale=[1.0, 1.0], size=(num_points, 2))
            structure_points.append(np.hstack([structure_xy, structure_z]))


        # Add some random floating debris/artifacts
        num_debris = 15
        for _ in range(num_debris):
            # Random position for floating debris
            center_x = np.random.uniform(self.bbox_min[0], self.bbox_max[0])
            center_y = np.random.uniform(self.bbox_min[1], self.bbox_max[1]) 
            center_z = np.random.uniform(10, 30) # Float between structures
            
            # Generate small cluster of points
            num_debris_points = self.n_points//(4*num_debris)
            debris_points = np.random.normal(
                loc=[center_x, center_y, center_z],
                scale=[0.5, 0.5, 0.5], 
                size=(num_debris_points, 3)
            )
            structure_points.append(debris_points)
            
        # Add some vegetation-like clusters near ground
        num_vegetation = 8
        for _ in range(num_vegetation):
            # Random position near ground
            base_x = np.random.uniform(self.bbox_min[0], self.bbox_max[0])
            base_y = np.random.uniform(self.bbox_min[1], self.bbox_max[1])
            
            # Generate cone-like point distribution
            num_veg_points = self.n_points//(4*num_vegetation)
            heights = np.random.triangular(5, 8, 12, num_veg_points).reshape(-1,1)
            spread = np.linspace(1.5, 0.1, num_veg_points).reshape(-1,1)
            xy_points = np.random.normal(loc=[base_x, base_y], scale=spread, size=(num_veg_points, 2))
            vegetation_points = np.hstack([xy_points, heights])
            structure_points.append(vegetation_points)

        # Add box-like volume artifacts
        num_boxes = 10
        for _ in range(num_boxes):
            # Random box center position
            center_x = np.random.uniform(self.bbox_min[0], self.bbox_max[0])
            center_y = np.random.uniform(self.bbox_min[1], self.bbox_max[1])
            center_z = np.random.uniform(10, 25)  # Float between structures
            
            # Random box dimensions
            box_width = np.random.uniform(5, 10)
            box_length = np.random.uniform(5, 10) 
            box_height = np.random.uniform(5, 10)
            
            # Generate points within box volume
            num_box_points = self.n_points//(4*num_boxes)
            box_x = np.random.uniform(center_x - box_width/2, center_x + box_width/2, num_box_points)
            box_y = np.random.uniform(center_y - box_length/2, center_y + box_length/2, num_box_points)
            box_z = np.random.uniform(center_z - box_height/2, center_z + box_height/2, num_box_points)
            
            box_points = np.column_stack([box_x, box_y, box_z])
            structure_points.append(box_points)

        return structure_points
        
    def generate_point_cloud(self):
        # Generate ground and structure points
        ground_points = self.generate_ground_points()
        structure_points = self.generate_structure_points()
        
        # Combine points
        points = np.vstack([ground_points] + structure_points)
        
        # Convert to cupoch PointCloud
        pcd = cph.geometry.PointCloud()
        pcd.points = cph.utility.Vector3fVector(points.astype(np.float64))
        pcd.crop(self.scene_bbox)
        points = np.asarray(pcd.points.cpu())
        return pcd, points
    
    def generate_voxel_grid(self, pcd):
        # Create a voxel grid from the point cloud
        voxel_grid = cph.geometry.VoxelGrid.create_from_point_cloud_within_bounds(
            pcd,
            voxel_size=self.voxel_size,
            min_bound=self.min_bound,
            max_bound=self.max_bound
        )

        return voxel_grid

    def point_to_voxel_index(self, point):
        """
        Convert a 3D point to its corresponding voxel index.
        
        Parameters:
        -----------
        point : array-like
            3D point coordinates [x, y, z]
            
        Returns:
        --------
        tuple
            (i, j, k) voxel grid indices
        """
        point = np.asarray(point)
        voxel_indices = np.floor((point - self.min_bound) / self.voxel_size).astype(int)
        return tuple(voxel_indices)


# Example usage:
generator = DatasetGenerator()
visualizer = PointCloudVisualizer()

# Generate data
pcd, points = generator.generate_point_cloud()
voxel_grid = generator.generate_voxel_grid(pcd)

# Visualize results
# visualizer.visualize_points(points)
# visualizer.visualize_voxel_grid(voxel_grid)

In [ ]:
visualizer.visualize_voxel_grid(voxel_grid)

In [40]:
import math

@cuda.jit(device=True)
def get_vector(result_vec, vec, idx):
    result_vec[0] = vec[idx + 0]
    result_vec[1] = vec[idx + 1]
    result_vec[2] = vec[idx + 2]

@cuda.jit(device=True)
def init_vector(result_vec):
    result_vec[0] = 0.0
    result_vec[1] = 0.0
    result_vec[2] = 0.0

@cuda.jit(device=True)
def add_vectors(result_vec, vec1, vec2):
    result_vec[0] = vec1[0] + vec2[0]
    result_vec[1] = vec1[1] + vec2[1]
    result_vec[2] = vec1[2] + vec2[2]

@cuda.jit(device=True)
def subtract_vectors(result_vec, vec1, vec2):
    result_vec[0] = vec1[0] - vec2[0]
    result_vec[1] = vec1[1] - vec2[1]
    result_vec[2] = vec1[2] - vec2[2]

@cuda.jit(device=True)
def dot_vectors(vec1, vec2):
    product = np.zeros(3, dtype=np.float32)
    product[0] = vec1[0] * vec2[0]
    product[1] = vec1[1] * vec2[1]
    product[2] = vec1[2] * vec2[2]
    return product[0] + product[1] + product[2]

@cuda.jit(device=True) 
def cross_vectors(result_vec, vec1, vec2):
    result_vec[0] = vec1[1] * vec2[2] - vec1[2] * vec2[1]
    result_vec[1] = vec1[2] * vec2[0] - vec1[0] * vec2[2]
    result_vec[2] = vec1[0] * vec2[1] - vec1[1] * vec2[0]

@cuda.jit(device=True)
def norm(vec):
    return math.sqrt(vec[0] * vec[0] + vec[1] * vec[1] + vec[2] * vec[2])

@cuda.jit(device=True)
def copy_vector(result_vec, vec, idx):
    result_vec[idx + 0] = vec[0]
    result_vec[idx + 1] = vec[1]
    result_vec[idx + 2] = vec[2]

@cuda.jit(device=True)
def normalize_vector(result_vec, original_vec):
    original_vec_mag = math.sqrt(original_vec[0] * original_vec[0] + original_vec[1] * original_vec[1] + original_vec[2] * original_vec[2])
    if original_vec_mag == 0.0:
        result_vec[0] = 0.0
        result_vec[1] = 0.0
        result_vec[2] = 0.0
    else:
        result_vec[0] = original_vec[0] / original_vec_mag
        result_vec[1] = original_vec[1] / original_vec_mag
        result_vec[2] = original_vec[2] / original_vec_mag

@cuda.jit(device=True)
def scale_vector(result_vec, original_vec, scalar):
    result_vec[0] = original_vec[0] * scalar
    result_vec[1] = original_vec[1] * scalar
    result_vec[2] = original_vec[2] * scalar


@cuda.jit 
def compute_obstacle_heuristic_force_at_anchor(
    anchor, masses, num_masses, mass_radius, detect_radius, force_result):
    # computes force contribution of each mass at anchor
    idx = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x

    if idx >= num_masses:
        return

    mass_vector = cuda.local.array(3, dtype=np.float32)
    get_vector(mass_vector, masses, 3*idx)

    # calculate distance and distance vector
    distance_vector = cuda.local.array(3, dtype=np.float32)
    subtract_vectors(distance_vector, mass_vector, anchor)
    distance = 0.0
    distance = norm(distance_vector)


    # compute if radius touches detect radius
    if distance - mass_radius < detect_radius:
        force_vector = cuda.local.array(3, dtype=np.float32)
        distance_unit_vector = cuda.local.array(3, dtype=np.float32)
        
        # heuristic goes here
        normalize_vector(distance_unit_vector, distance_vector)
        force_magnitude = 1.0 / (distance * distance) if distance > 0.001 else 1000.0
        force_magnitude = 1.0*force_magnitude
        scale_vector(force_vector, distance_unit_vector, force_magnitude)
        # heuristic end

        copy_vector(force_result, force_vector, 3*idx)

    else:
        force_vector = cuda.local.array(3, dtype=np.float32)
        init_vector(force_vector)
        copy_vector(force_result, force_vector, 3*idx)


In [ ]:
masses = points.flatten()
num_masses = points.shape[0]
mass_radius = 1.0
detect_radius = 50.0
d_masses = cuda.to_device(masses)
threads_per_block = 256
blocks_per_grid = (num_masses + threads_per_block - 1) // threads_per_block

anchor = np.array([0.0, 0.0, 0.0], dtype=np.float32)
d_anchor = cuda.to_device(anchor)
force_result = np.zeros(len(masses), dtype=np.float32)
d_force_result = cuda.to_device(force_result)

In [ ]:
# time take to load into memory
start = time.time()

masses = points.flatten()
num_masses = points.shape[0]
mass_radius = 1.0
detect_radius = 50.0
d_masses = cuda.to_device(masses)
threads_per_block = 256
blocks_per_grid = (num_masses + threads_per_block - 1) // threads_per_block

anchor = np.array([0.0, 0.0, 0.0], dtype=np.float32)
d_anchor = cuda.to_device(anchor)
force_result = np.zeros(len(masses), dtype=np.float32)
d_force_result = cuda.to_device(force_result)

end = time.time()
print(f"Time taken to load into memory: {end - start} seconds")

start = time.time()
compute_obstacle_heuristic_force_at_anchor[blocks_per_grid, threads_per_block](d_anchor, d_masses, num_masses, mass_radius, detect_radius, d_force_result)
end = time.time()
print(f"Time taken to compute force: {end - start} seconds")

start = time.time()
force_result = d_force_result.copy_to_host()
end = time.time()
print(f"Time taken to copy to host: {end - start} seconds")


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Create arrays for x, y, z coordinates and force magnitudes
x = []
y = []
z = []
magnitudes = []

# Extract coordinates and calculate force magnitudes
for (i, j, k), force in forces.items():
    x.append(i)
    y.append(j) 
    z.append(k)
    magnitudes.append(np.linalg.norm(force))

# Create 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y, 
    z=z,
    mode='markers',
    marker=dict(
        size=5,
        color=magnitudes,
        colorscale='Viridis',
        colorbar=dict(title='Force Magnitude'),
        opacity=0.8
    ),
    hovertemplate='x: %{x}<br>y: %{y}<br>z: %{z}<br>magnitude: %{marker.color:.2f}<extra></extra>'
)])

# Update layout
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title='3D Force Field Visualization',
    margin=dict(l=0, r=0, t=30, b=0)
)

fig.show()


In [ ]:
visualizer.visualize_voxel_grid(voxel_grid)

In [ ]:
import math
import numpy as np
import time
from numba import cuda, float32

# -------------------------------
# Optimized kernel for batched anchors
# -------------------------------
@cuda.jit
def compute_forces_for_anchors(anchors, masses, n_masses, mass_radius, detect_radius, forces):
    """
    Each block computes the net force at one anchor.
    Each thread in the block sums the contributions of a subset of masses.
    Then a shared-memory reduction is performed.
    """
    anchor_idx = cuda.blockIdx.x  # each block handles one anchor
    tid = cuda.threadIdx.x

    # Return if anchor index exceeds number of anchors
    if anchor_idx >= anchors.shape[0]:
        return

    # Allocate shared memory for partial sums (assumes threads_per_block is 256)
    sdata_x = cuda.shared.array(256, dtype=float32)
    sdata_y = cuda.shared.array(256, dtype=float32)
    sdata_z = cuda.shared.array(256, dtype=float32)

    # Load the anchor coordinates (each anchor is a 3-element vector)
    ax = anchors[anchor_idx, 0]
    ay = anchors[anchor_idx, 1]
    az = anchors[anchor_idx, 2]

    # Each thread computes a partial sum over a grid-stride loop of masses.
    partial_x = 0.0
    partial_y = 0.0
    partial_z = 0.0

    # Loop over masses assigned to this thread
    for m in range(tid, n_masses, cuda.blockDim.x):
        # Get mass coordinates (assuming masses is shape (n_masses, 3))
        mx = masses[m, 0]
        my = masses[m, 1]
        mz = masses[m, 2]

        # Compute distance vector (mass - anchor)
        dx = mx - ax
        dy = my - ay
        dz = mz - az
        dist = math.sqrt(dx * dx + dy * dy + dz * dz)

        # Check if the mass is within the detect radius (adjusted for mass radius)
        if dist - mass_radius < detect_radius:
            # Compute force magnitude using the heuristic (avoid division by zero)
            if dist > 0.001:
                inv_dist = 1.0 / dist
                fmag = 1.0 / (dist * dist)
            else:
                inv_dist = 0.0  # When too close, direction is undefined; use fmag directly.
                fmag = 1000.0
            # Compute unit vector and then scale by the force magnitude
            fx = (dx * inv_dist) * fmag
            fy = (dy * inv_dist) * fmag
            fz = (dz * inv_dist) * fmag

            # Accumulate partial force contributions
            partial_x += fx
            partial_y += fy
            partial_z += fz

    # Store the partial results into shared memory
    sdata_x[tid] = partial_x
    sdata_y[tid] = partial_y
    sdata_z[tid] = partial_z
    cuda.syncthreads()

    # Parallel reduction in shared memory to sum up all partial contributions
    stride = cuda.blockDim.x // 2
    while stride > 0:
        if tid < stride:
            sdata_x[tid] += sdata_x[tid + stride]
            sdata_y[tid] += sdata_y[tid + stride]
            sdata_z[tid] += sdata_z[tid + stride]
        cuda.syncthreads()
        stride //= 2

    # Write the final result for this anchor
    if tid == 0:
        forces[anchor_idx, 0] = sdata_x[0]
        forces[anchor_idx, 1] = sdata_y[0]
        forces[anchor_idx, 2] = sdata_z[0]


# -------------------------------
# Host-side setup and kernel launch
# -------------------------------

# Example: Create anchors over a 50x50x50 grid
n_dim = 50
n_anchors = n_dim * n_dim * n_dim
anchors = np.empty((n_anchors, 3), dtype=np.float32)
idx = 0
for i in range(n_dim):
    for j in range(n_dim):
        for k in range(n_dim):
            anchors[idx, :] = np.array([float(i), float(j), float(k)], dtype=np.float32)
            idx += 1

# 'points' should be your array of masses with shape (num_masses, 3)
# For this example, let's assume you have a variable `points` already defined.
# If your masses are originally stored in a flattened array, reshape them:
# masses = points.reshape(-1, 3)
masses = points.reshape(-1, 3)  # adjust as necessary
num_masses = masses.shape[0]

mass_radius = 1.0
detect_radius = 50.0

# Transfer data to the device only once
d_anchors = cuda.to_device(anchors)
d_masses = cuda.to_device(masses)
# Prepare device array for the computed forces for each anchor
d_forces = cuda.device_array((n_anchors, 3), dtype=np.float32)

# Choose threads per block (must match the size of shared arrays in the kernel)
threads_per_block = 256
# One block per anchor
blocks_per_grid = n_anchors

# Launch the kernel and time its execution
start = time.time()
compute_forces_for_anchors[blocks_per_grid, threads_per_block](
    d_anchors, d_masses, num_masses, mass_radius, detect_radius, d_forces
)
# Make sure to synchronize to get accurate timing
cuda.synchronize()
end = time.time()
print(f"Kernel execution time: {end - start} seconds")

# Copy the result back to the host (only one array with one 3-vector per anchor)
forces = d_forces.copy_to_host()

# 'forces' now is an (n_anchors, 3) array containing the net force at each anchor.


In [ ]:
import plotly.graph_objects as go

# Calculate magnitude of forces
force_magnitudes = np.linalg.norm(forces, axis=1)

# Create 3D scatter plot
fig = go.Figure(data=go.Volume(
    x=anchors[:, 0],
    y=anchors[:, 1], 
    z=anchors[:, 2],
    value=force_magnitudes,
    isomin=force_magnitudes.min(),
    isomax=force_magnitudes.max(),
    opacity=0.1,
    surface_count=50,
    colorscale='Turbo'
))

# Update layout
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title='3D Force Field Visualization',
    margin=dict(l=0, r=0, t=30, b=0)
)

fig.show()